In [1]:
# important: please modify the path as your local spark location 
spark_path = "D:/spark-2.3.1-bin-hadoop2.7"

In [2]:
# configure spark, a message regrads time will print out if success
import findspark
from datetime import datetime
findspark.init("D:/spark-2.3.1-bin-hadoop2.7")
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("ibm_hr_simple")
sc = SparkContext(conf = conf)
print("spark is well set at " + str(datetime.now()))

spark is well set at 2018-08-01 10:03:07.645657


In [3]:
# open spark session
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)
spark

In [4]:
# load dataset
ibm_hr = spark.read.csv("../data/WA_Fn-UseC_-HR-Employee-Attrition.csv", header=True, mode="DROPMALFORMED")
ibm_hr.show(3)

+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
# define function to transform boolean
def bool_to_int(b):
    if b == "Yes":
        return 1
    else:
        return 0
# register user defined function with spark SQL
udf_bool_to_int = udf(bool_to_int, IntegerType())
# add column
ibm_hr_target = ibm_hr.withColumn("Attrition_numerical", udf_bool_to_int("Attrition"))
# check the result
ibm_hr_target.select("Attrition", "Attrition_numerical").show(3)

+---------+-------------------+
|Attrition|Attrition_numerical|
+---------+-------------------+
|      Yes|                  1|
|       No|                  0|
|      Yes|                  1|
+---------+-------------------+
only showing top 3 rows



In [6]:
ibm_hr_target = ibm_hr_target.drop("Attrition")
ibm_hr_target.count(), len(ibm_hr_target.columns)

(1470, 35)

In [7]:
categorical = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
numerical = [u'Age', u'DailyRate', u'DistanceFromHome', u'Education', u'EmployeeNumber', u'EnvironmentSatisfaction', \
             u'HourlyRate', u'JobInvolvement', u'JobLevel', u'JobSatisfaction', \
             u'MonthlyIncome', u'MonthlyRate', u'NumCompaniesWorked', \
             u'PercentSalaryHike', u'PerformanceRating', u'RelationshipSatisfaction', \
             u'StockOptionLevel', u'TotalWorkingYears', \
             u'TrainingTimesLastYear', u'WorkLifeBalance', u'YearsAtCompany', \
             u'YearsInCurrentRole', u'YearsSinceLastPromotion', u'YearsWithCurrManager']
len(categorical), len(numerical)

(7, 24)

In [8]:
# use pandas to get_dummies
import pandas as pd
pd_cat = pd.get_dummies(ibm_hr_target.select(categorical).toPandas())
pd_cat.head(3)

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
1,0,1,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [9]:
ibm_hr_cat = spark.createDataFrame(pd_cat)
for c in ibm_hr_cat.columns:
    ibm_hr_cat = ibm_hr_cat.withColumn(c, ibm_hr_cat[c].cast(IntegerType()))
ibm_hr_att = ibm_hr_target.select("Attrition_numerical")
ibm_hr_target = ibm_hr_target.select(numerical)
for c in ibm_hr_target.columns:
    ibm_hr_target = ibm_hr_target.withColumn(c, ibm_hr_target[c].cast(IntegerType()))

In [10]:
ibm_hr_target = ibm_hr_target.join(ibm_hr_cat)
ibm_hr_target = ibm_hr_target.join(ibm_hr_att)
ibm_hr_target.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- DailyRate: integer (nullable = true)
 |-- DistanceFromHome: integer (nullable = true)
 |-- Education: integer (nullable = true)
 |-- EmployeeNumber: integer (nullable = true)
 |-- EnvironmentSatisfaction: integer (nullable = true)
 |-- HourlyRate: integer (nullable = true)
 |-- JobInvolvement: integer (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: integer (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- MonthlyRate: integer (nullable = true)
 |-- NumCompaniesWorked: integer (nullable = true)
 |-- PercentSalaryHike: integer (nullable = true)
 |-- PerformanceRating: integer (nullable = true)
 |-- RelationshipSatisfaction: integer (nullable = true)
 |-- StockOptionLevel: integer (nullable = true)
 |-- TotalWorkingYears: integer (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: integer (nullable = true)
 |-- YearsAtCompany: integer (nullable = true

In [11]:
feature_col = ibm_hr_target.columns
feature_col.remove("Attrition_numerical")
len(feature_col), len(ibm_hr_target.columns)

(52, 53)

### The problem here seems to be with the join
to have a clearer view, I'll switch to another notebook

In [31]:
ibm_hr_cat.count()

1470

In [32]:
ibm_hr_att.count()

1470

In [30]:
ibm_hr_cat.join(ibm_hr_att).count()

AnalysisException: 'Detected implicit cartesian product for INNER join between logical plans\nProject\n+- LogicalRDD [BusinessTravel_Non-Travel#360L, BusinessTravel_Travel_Frequently#361L, BusinessTravel_Travel_Rarely#362L, Department_Human Resources#363L, Department_Research & Development#364L, Department_Sales#365L, EducationField_Human Resources#366L, EducationField_Life Sciences#367L, EducationField_Marketing#368L, EducationField_Medical#369L, EducationField_Other#370L, EducationField_Technical Degree#371L, Gender_Female#372L, Gender_Male#373L, JobRole_Healthcare Representative#374L, JobRole_Human Resources#375L, JobRole_Laboratory Technician#376L, JobRole_Manager#377L, JobRole_Manufacturing Director#378L, JobRole_Research Director#379L, JobRole_Research Scientist#380L, JobRole_Sales Executive#381L, JobRole_Sales Representative#382L, MaritalStatus_Divorced#383L, ... 4 more fields], false\nand\nProject\n+- Relation[Age#10,Attrition#11,BusinessTravel#12,DailyRate#13,Department#14,Dis

In [23]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer
assembler = VectorAssembler(inputCols = feature_col, outputCol = "features")
assembled = assembler.transform(ibm_hr_target)
(training_data, testing_data) = assembled.randomSplit([0.8, 0.2], seed = 13234)
training_data.count(), test_data.count()

AnalysisException: 'Detected implicit cartesian product for INNER join between logical plans\nJoin Inner\n:- Join Inner\n:  :- Join Inner\n:  :  :- Project [cast(Age#10 as int) AS Age#691, cast(DailyRate#13 as int) AS DailyRate#716, cast(DistanceFromHome#15 as int) AS DistanceFromHome#741, cast(Education#16 as int) AS Education#766, cast(EmployeeNumber#19 as int) AS EmployeeNumber#791, cast(EnvironmentSatisfaction#20 as int) AS EnvironmentSatisfaction#816, cast(HourlyRate#22 as int) AS HourlyRate#841, cast(JobInvolvement#23 as int) AS JobInvolvement#866, cast(JobLevel#24 as int) AS JobLevel#891, cast(JobSatisfaction#26 as int) AS JobSatisfaction#916, cast(MonthlyIncome#28 as int) AS MonthlyIncome#941, cast(MonthlyRate#29 as int) AS MonthlyRate#966, cast(NumCompaniesWorked#30 as int) AS NumCompaniesWorked#991, cast(PercentSalaryHike#33 as int) AS PercentSalaryHike#1016, cast(PerformanceRating#34 as int) AS PerformanceRating#1041, cast(RelationshipSatisfaction#35 as int) AS RelationshipS